In [15]:
#Load libraries
import os
import numpy as np
import pandas as pd
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
import matplotlib.pyplot as plt
from PIL import Image, ImageOps

from torch.utils.data.dataset import Dataset
import torchvision.transforms as transforms
import cv2

In [16]:
# from google.colab import drive
# drive.mount('/content/drive')

In [17]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

In [18]:
#Transforms
transformer=transforms.Compose([
    #transforms.RandomHorizontalFlip(),
    transforms.Resize((256,256)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5]),
    transforms.Grayscale(num_output_channels=1)
])

In [19]:
#Dataloader

#Path for training and testing directory
train_path='../input/wo-other-odir-clahe-augmented/data_5/train'
validation_path='../input/wo-other-odir-clahe-augmented/data_5/valid'
test_path='../input/wo-other-odir-clahe-augmented/data_5/test'

# train_path='/content/drive/MyDrive/data_5/train'
# validation_path='/content/drive/MyDrive/data_5/valid'
# test_path='/content/drive/MyDrive/data_5/test'



train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)

validation_loader=DataLoader(
    torchvision.datasets.ImageFolder(validation_path,transform=transformer),
    batch_size=64, shuffle=True
)

test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [20]:
#CNN Network


# class ConvNet(nn.Module):
#     def __init__(self,num_classes):
#         super(ConvNet,self).__init__()
        
#         #Output size after convolution filter
#         #((w-f+2P)/s) +1
        
#         #Input shape= (256,3,150,150)
        
#         self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
#         #Shape= (256,12,150,150)
#         self.bn1=nn.BatchNorm2d(num_features=12)
#         #Shape= (256,12,150,150)
#         self.relu1=nn.ReLU()
#         #Shape= (256,12,150,150)
        
#         self.pool=nn.MaxPool2d(kernel_size=2)
#         #Reduce the image size be factor 2
#         #Shape= (256,12,75,75)
        
        
#         self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
#         #Shape= (256,20,75,75)
#         self.relu2=nn.ReLU()
#         #Shape= (256,20,75,75)
        
        
        
#         self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
#         #Shape= (256,32,75,75)
#         self.bn3=nn.BatchNorm2d(num_features=32)
#         #Shape= (256,32,75,75)
#         self.relu3=nn.ReLU()
#         #Shape= (256,32,75,75)
        
        
#         self.fc=nn.Linear(in_features=128 * 128 * 32,out_features=num_classes)
        
        
        
#         #Feed forwad function
        
#     def forward(self,input):
#         output=self.conv1(input)
#         output=self.bn1(output)
#         output=self.relu1(output)
            
#         output=self.pool(output)
            
#         output=self.conv2(output)
#         output=self.relu2(output)
            
#         output=self.conv3(output)
#         output=self.bn3(output)
#         output=self.relu3(output)
            
            
#             #Above output will be in matrix form, with shape (256,32,75,75)
            
#         output=output.view(-1,32*128*128)
            
            
#         output=self.fc(output)
            
#         return output

In [21]:
#CNN Network modified


class ConvNet(nn.Module):
    def __init__(self,num_classes):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=32)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)

        self.pool1=nn.MaxPool2d(kernel_size=2,stride=2)
         
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=32,out_channels=128,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.bn2=nn.BatchNorm2d(num_features=128)
        self.relu2=nn.Softmax(dim=1)
        self.pool2=nn.MaxPool2d(kernel_size=2,stride=2)
        #Shape= (256,20,75,75)
        
        
        
        # self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        # #Shape= (256,32,75,75)
        # self.bn3=nn.BatchNorm2d(num_features=32)
        # #Shape= (256,32,75,75)
        # self.relu3=nn.Softmax(dim=1)
        # #Shape= (256,32,75,75)

       
        
        
        self.fc=nn.Linear(in_features=128* 64 * 64,out_features=128)
        self.fc2=nn.Linear(128,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
        output=self.pool1(output)

            
        output=self.conv2(output)
        output=self.bn2(output)
        output=self.relu2(output)
        output=self.pool2(output)
            
        # output=self.conv3(output)
        # output=self.bn3(output)
        # output=self.relu3(output)
       
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,128*64*64)
            
            
        output=self.fc(output)
        output=self.fc2(output)
            
        return output


In [22]:
model=ConvNet(num_classes=4).to(device)


#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.002,weight_decay=0.0003)
loss_function=nn.CrossEntropyLoss()

num_epochs=10

# calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))
validation_count=len(glob.glob(validation_path+'/**/*.jpg'))

print(train_count,test_count,validation_count)

In [23]:
#Model training and saving best model

iter = []
train_losses = []
valid_losses = []
itr = 0
total_step = len(train_loader)

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        itr = itr + 1
        if (i+1) % 20 == 0:
            iter.append(itr)
            train_losses.append(loss.item())
            print ('Epoch [{}/{}], Step [{}/{}], Training Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count


    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy))



    
    
    
    model.eval()
    validation_accuracy=0.0
    validation_loss=0.0
    
    total_step = len(validation_loader)
    
    for i, (images,labels) in enumerate(validation_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        
#         itr = itr + 1
        if (i+1) % 20 == 0:
            iter.append(itr)
            valid_losses.append(loss.item())
            print ('Epoch [{}/{}], Step [{}/{}], Validation Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        
        validation_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        validation_accuracy+=int(torch.sum(prediction==labels.data))
        
    validation_accuracy=validation_accuracy/validation_count
    validation_loss=validation_loss/validation_count

    print('Epoch: '+str(epoch)+' Validation Loss: '+str(validation_loss)+' Validation Accuracy: '+str(validation_accuracy))

In [24]:
y_pred = []
y_test = []

# Evaluation on testing dataset
model.eval()


test_accuracy=0.0

with torch.no_grad():
    correct = 0
    total = 0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
        
        total += labels.size(0)
        correct += (prediction == labels).sum().item()

        y_pred.extend(prediction)
        y_test.extend(labels)

test_accuracy=test_accuracy/test_count


print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))


#Save the best model
if test_accuracy>best_accuracy:
    torch.save(model.state_dict(),'./best_checkpoint.model')
    best_accuracy=test_accuracy

In [27]:
plt.plot(iter[ : 60], train_losses, label='Loss')
plt.title('Loss Curve')
plt.ylabel('Loss')
plt.xlabel('Iteration')
plt.legend()

In [28]:
yy_test = []

for i in range(len(y_test)):
    yy_test.append(int(y_test[i]))

print(yy_test)

yy_pred = []

for i in range(len(y_pred)):
    yy_pred.append(int(y_pred[i]))

print(yy_pred)

In [29]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns

def show_confusion_matrix(confusion_matrix):
    hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
    hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
    hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=0, ha='center')
    plt.ylabel('True Labels')
    plt.xlabel('Predicted Labels');
cm = confusion_matrix(yy_test, yy_pred)
df_cm = pd.DataFrame(cm, index=['AMD', 'Cataract', 'Myopia', 'Normal'], columns=['AMD', 'Cataract', 'Myopia', 'Normal'])
show_confusion_matrix(df_cm)

In [30]:
print('F1 score: ', f1_score(yy_test, yy_pred, average='weighted'))
print('Precision: ', precision_score(yy_test, yy_pred, average='weighted'))
print('Recall: ', recall_score(yy_test, yy_pred, average='weighted'))